Analysis

In [63]:
import itertools
import numpy as np
def get_permutations(l,n=3):
    prmts=[]
    for p in itertools. permutations(l, n):
        if sorted(p) not in prmts:
            prmts.append(sorted(p))
    return [ np.asarray(p) for p in prmts if isinstance(p,list) ]
def jsonKeys2int(x):
    if isinstance(x, dict):
            return {int(k):v for k,v in x.items()}
    return x

In [64]:
import pandas as pd

In [65]:
cl=pd.read_json('cl.json').reset_index(drop=True)
cl['sltn']=cl['sltn'].apply(jsonKeys2int)
cl=cl.sort_values('n').reset_index(drop=True)
cl6=pd.read_json('cl6.json').reset_index(drop=True)
cl6['sltn']=cl6['sltn'].apply(jsonKeys2int)
cl6=cl6.sort_values('n').reset_index(drop=True)

DD analysis

In [66]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth',500)

In [67]:
def extract(l,x,νmax=5):
    for i in range(νmax):
        try:
            l.remove(x)
        except:
            break
    return l

def remove_list(l,pr):
    for x in pr:
        try:
            l=extract(l,x)
        except:
            pass
    return l

def remove(l,pr):
    if pr:
        pr=np.concatenate(pr).ravel()
    l=remove_list(l,pr)
    return l

def get_massless_fermions(l,ν,m=0,Dim=5):
    '''
    Zero determinant check implemented.
    '''
    if Dim==5:
        d=1
    elif Dim==6:
        d=2
    s=-(ν+m)//d
    l=extract(l,ν)
    lold=l.copy()
    if m:
        l=extract(l,m)
    #Dirac masses
    det0=0
    for ss in [s,-s]:
        pr=[ p for  p in get_permutations( l,n=2  ) if np.append( p,ss).sum()==0 ]
        for i in range(len(pr)):
            if pr and sum( [ lold.count(x) for x in pr[i] ] )%2!=0:
                det0=det0+1
            l=remove(l,pr)
            if det0!=0:
                l.append(-1000)

    #Majorana masses
    for ss in [s,-s]:
        pr=[ll for ll in l if 2*ll+ss==0]
        l=remove_list(l,pr)
    #If repeated an extra S' gives at least degenerate Majorana mass
    return list(np.unique(l))
    #return l


def count_massless_fermions(l,ν,m=0,Dim=5):
    return len( get_massless_fermions(l,ν,m,Dim=Dim) )

def get_label(model,Dim=5):
    if Dim==5:
        dd=''
        dm=1
    elif Dim==6:
        dd='2'
        dm=2    
    if model=='DarkDirac':
        label='D→nu_R+{}fi+{}fj'.format(dd,dd)
    elif model=='DarkMajor':
        label='D→nu_R+{}fi'.format(2*dm)
    elif model=='XDirac':
        label='X→nu_R+{}fi+{}fj'.format(dd,dd)
    else:
        label='X→nu_R+{}fi'.format(2*dm)
    return model,label

def sltn_intersection(row,model='DarkDirac',Dim=5):
    model,label=get_label(model,Dim=Dim)
    sltn_int=0
    l=list(itertools.chain(*row['sltn'].values() ) )
    #print(l,label)
    for d in row['nu_R']:
        if d.get(label):
            #print(d)
            intrs=len(np.intersect1d(d.get(label), l ))
            if intrs>sltn_int:
                sltn_int=intrs
    return sltn_int

In [68]:
assert get_massless_fermions([2, -3, -3, -5, 9, -13, 15, 16, -18],ν=-3)==[9]
assert count_massless_fermions( [2, -5, -5, -5, 7, 8, 8, -10],ν=8,m=0)==2
assert count_massless_fermions( [2, -5, -5, -5, 7, 8, 8, -10],ν=8,m=-5)==1
assert count_massless_fermions( [2, -3, -10, 13, 13, -15], ν=13)==0
assert count_massless_fermions( [5, 13, 13, -16, -18, -23, 26],ν=13)==3
assert get_massless_fermions( [2, -4, 7, 9, -18, -18, 22],ν=-18 )==[2,7]
# XDirac with two triplets and Dim 6
assert get_massless_fermions([2, -7, -8, -8, -8, 18, 18, 18, -25],-8,m=18,Dim=6)==[-25]
#Check determinant zero mass matrix
assert get_massless_fermions( [5, 5, 7, -8, -12, -12, 15],ν=-12,m=0,Dim=5  )==[-1000, -8, 15]
assert get_massless_fermions([2, 4, 4, -15, -17, 26, 26, -30],ν=26,m=0,Dim=6)==[-1000, -30]

In [69]:
def massfer(row,label='DarkDirac',nmax=10,Dim=5):        
    # 
    if label=='DarkDirac' or label=='DarkMajor':
        if label=='DarkDirac':
            nuR='DD'
        else:
            nuR='DM'
        if row.get(label)>0:
            cdd=[]
            for ν in row.get(nuR):
                l=row.get('solution').copy()
                cdd.append( count_massless_fermions(l , ν,Dim=Dim)  )
            return min(cdd)
        else:
            return nmax
    if label=='XDirac' or label=='XMajor':
        if label=='XDirac':
            nuR='XD'
        else:
            nuR='XM'
        if row.get(label)>0:
            cddmax=100
            for m in row['sltn'].get(3):
                cdp=[]
                for ν in row.get(nuR):
                    l=row.get('solution').copy()
                    l=[ll for ll in l if ll!=m]
                    cdp.append( count_massless_fermions(l , ν,m,Dim=Dim)  )
                if min(cdp)<cddmax:
                    cdd=cdp
            return min(cdd)
        else:
            return nmax    

In [70]:
cl['DD_massless']=cl.apply(lambda row:   massfer(row,label='DarkDirac'),axis='columns')
cl['DM_massless']=cl.apply(lambda row:   massfer(row,label='DarkMajor'),axis='columns')
cl['XD_massless']=cl.apply(lambda row:   massfer(row,label='XDirac'   ),axis='columns')
cl['XM_massless']=cl.apply(lambda row:   massfer(row,label='XMajor'   ),axis='columns')

In [71]:
cl6['DD_massless']=cl6.apply(lambda row: massfer(row,label='DarkDirac',Dim=6),axis='columns')
cl6['DM_massless']=cl6.apply(lambda row: massfer(row,label='DarkMajor',Dim=6),axis='columns')
cl6['XD_massless']=cl6.apply(lambda row: massfer(row,label='XDirac'   ,Dim=6),axis='columns')
cl6['XM_massless']=cl6.apply(lambda row: massfer(row,label='XMajor'   ,Dim=6),axis='columns')

In [72]:
z=np.array([1,1,1,1,1,1,-4,-4,-4,-4,5,5])
z.sum(),(z**3).sum()

(0, 0)

In [73]:
cl['total']   =cl['solution'].reset_index(drop=True).apply(lambda l: np.abs( np.array(l) ).sum() )
#cl['sltn_len']=-cl['sltn'].apply(lambda d: list(itertools.chain(*d.values()))).apply(len)

In [74]:
cl6['total']   =cl6['solution'].reset_index(drop=True).apply(lambda l: np.abs( np.array(l) ).sum() )
#cl6['sltn_len']=-cl6['sltn'].apply(lambda d: list(itertools.chain(*d.values()))).apply(len)

In [75]:
row=cl[cl['solution'].astype(str)=='[1, 4, 5, 5, -8, -10, -10, 13]'].reset_index(drop=True).loc[0].to_dict()

In [76]:
sltn_intersection(row,model='DarkMajor')

2

## Filter unique solutions

Pure solutions

In [77]:
dim=[5,6]
#d=dim[0]
for d in dim:
    if d==5:
        fl=pd.DataFrame()
        clc=cl.copy()
    else:
        fl6=pd.DataFrame()
        clc=cl6.copy()
    models=pd.DataFrame([
         {'model':'DarkDirac','condition':((clc['DarkDirac']>0)  & (clc['DarkMajor']==0) & (clc['XDirac']==0) & (clc['XMajor']==0))},
         {'model':'DarkDirac','condition':((clc['DarkDirac']>0)  & (clc['DarkMajor']>0)  & (clc['XDirac']==0) & (clc['XMajor']==0))},
         {'model':'DarkDirac','condition':((clc['DarkDirac']>0)  & (clc['DarkMajor']==0) & (clc['XDirac']>0)  & (clc['XMajor']==0))},
         {'model':'DarkDirac','condition':((clc['DarkDirac']>0)  & (clc['DarkMajor']==0) & (clc['XDirac']==0) & (clc['XMajor']>0))},
         {'model':'DarkDirac','condition':((clc['DarkDirac']>0)  & (clc['DarkMajor']>0)  & (clc['XDirac']>0)  & (clc['XMajor']==0))},
         {'model':'DarkDirac','condition':((clc['DarkDirac']>0)  & (clc['DarkMajor']>0)  & (clc['XDirac']==0) & (clc['XMajor']>0))},
         {'model':'DarkDirac','condition':((clc['DarkDirac']>0)  & (clc['DarkMajor']==0) & (clc['XDirac']>0)  & (clc['XMajor']>0))},
         {'model':'DarkDirac','condition':((clc['DarkDirac']>0)  & (clc['DarkMajor']>0)  & (clc['XDirac']>0)  & (clc['XMajor']>0))},
         {'model':'DarkMajor','condition':((clc['DarkDirac']==0) & (clc['DarkMajor']>0)  & (clc['XDirac']==0) & (clc['XMajor']==0))},
         {'model':'DarkMajor','condition':((clc['DarkDirac']> 0) & (clc['DarkMajor']>0)  & (clc['XDirac']==0) & (clc['XMajor']==0))},
         {'model':'DarkMajor','condition':((clc['DarkDirac']==0) & (clc['DarkMajor']>0)  & (clc['XDirac']> 0) & (clc['XMajor']==0))},
         {'model':'DarkMajor','condition':((clc['DarkDirac']==0) & (clc['DarkMajor']>0)  & (clc['XDirac']==0) & (clc['XMajor']> 0))},
         {'model':'DarkMajor','condition':((clc['DarkDirac']> 0) & (clc['DarkMajor']>0)  & (clc['XDirac']> 0) & (clc['XMajor']==0))},
         {'model':'DarkMajor','condition':((clc['DarkDirac']> 0) & (clc['DarkMajor']>0)  & (clc['XDirac']==0) & (clc['XMajor']> 0))},
         {'model':'DarkMajor','condition':((clc['DarkDirac']==0) & (clc['DarkMajor']>0)  & (clc['XDirac']> 0) & (clc['XMajor']> 0))},
         {'model':'DarkMajor','condition':((clc['DarkDirac']> 0) & (clc['DarkMajor']>0)  & (clc['XDirac']> 0) & (clc['XMajor']> 0))},
         {'model':'XDirac'   ,'condition':((clc['DarkDirac']==0) & (clc['DarkMajor']==0) & (clc['XDirac']>0)  & (clc['XMajor']==0))},
         {'model':'XDirac'   ,'condition':((clc['DarkDirac']> 0) & (clc['DarkMajor']==0) & (clc['XDirac']>0)  & (clc['XMajor']==0))},
         {'model':'XDirac'   ,'condition':((clc['DarkDirac']==0) & (clc['DarkMajor']> 0) & (clc['XDirac']>0)  & (clc['XMajor']==0))},
         {'model':'XDirac'   ,'condition':((clc['DarkDirac']==0) & (clc['DarkMajor']==0) & (clc['XDirac']>0)  & (clc['XMajor']> 0))},
         {'model':'XDirac'   ,'condition':((clc['DarkDirac']> 0) & (clc['DarkMajor']> 0) & (clc['XDirac']>0)  & (clc['XMajor']==0))},
         {'model':'XDirac'   ,'condition':((clc['DarkDirac']> 0) & (clc['DarkMajor']==0) & (clc['XDirac']>0)  & (clc['XMajor']> 0))},
         {'model':'XDirac'   ,'condition':((clc['DarkDirac']==0) & (clc['DarkMajor']> 0) & (clc['XDirac']>0)  & (clc['XMajor']> 0))},
         {'model':'XDirac'   ,'condition':((clc['DarkDirac']> 0) & (clc['DarkMajor']> 0) & (clc['XDirac']>0)  & (clc['XMajor']> 0))},
         {'model':'XMajor'   ,'condition':((clc['DarkDirac']==0) & (clc['DarkMajor']==0) & (clc['XDirac']==0) & (clc['XMajor']>0))},
         {'model':'XMajor'   ,'condition':((clc['DarkDirac']> 0) & (clc['DarkMajor']==0) & (clc['XDirac']==0) & (clc['XMajor']>0))},
         {'model':'XMajor'   ,'condition':((clc['DarkDirac']==0) & (clc['DarkMajor']> 0) & (clc['XDirac']==0) & (clc['XMajor']>0))},
         {'model':'XMajor'   ,'condition':((clc['DarkDirac']==0) & (clc['DarkMajor']==0) & (clc['XDirac']> 0) & (clc['XMajor']>0))},
         {'model':'XMajor'   ,'condition':((clc['DarkDirac']> 0) & (clc['DarkMajor']> 0) & (clc['XDirac']==0) & (clc['XMajor']>0))},
         {'model':'XMajor'   ,'condition':((clc['DarkDirac']> 0) & (clc['DarkMajor']==0) & (clc['XDirac']> 0) & (clc['XMajor']>0))},
         {'model':'XMajor'   ,'condition':((clc['DarkDirac']==0) & (clc['DarkMajor']> 0) & (clc['XDirac']> 0) & (clc['XMajor']>0))},
         {'model':'XMajor'   ,'condition':((clc['DarkDirac']> 0) & (clc['DarkMajor']> 0) & (clc['XDirac']> 0) & (clc['XMajor']>0))}
    ])
    #i=0
    for i in models.index:
        mdl=models.loc[i,'model']
        tmp=clc[models.loc[i,'condition']].reset_index(drop=True)
        if not tmp.empty:
            tmp['sltn_len']=-tmp.apply(lambda row: sltn_intersection(row, model=mdl,Dim=d ) ,axis='columns')
            massless={'DarkDirac':'DD_massless','DarkMajor':'DM_massless','XDirac':'XD_massless','XMajor':'XM_massless'}
            if d==5:
                fl=fl.append(
                    tmp.sort_values(['n',massless[mdl],'sltn_len','total']
                     ).drop_duplicates(['n',massless[mdl],'sltn_len']).reset_index(drop=True)
                ).reset_index(drop=True)
                fl['str']=fl['solution'].astype(str)
                fl=fl.drop_duplicates('str').drop('str',axis='columns').reset_index(drop=True)                 
            if d==6:
                fl6=fl6.append(
                    tmp.sort_values(['n',massless[mdl],'sltn_len','total']
                     ).drop_duplicates(['n',massless[mdl],'sltn_len']).reset_index(drop=True)
                ).reset_index(drop=True)
                fl6['str']=fl6['solution'].astype(str)
                fl6=fl6.drop_duplicates('str').drop('str',axis='columns').reset_index(drop=True)                                 

In [78]:
cl[cl['solution'].astype(str)=='[5, 5, 7, -8, -12, -12, 15]']

,k,l,n,solution,gcd,nu_R,DarkDirac,DarkMajor,XDirac,XMajor,sltn,DD,DM,XD,XM,DD_massless,DM_massless,XD_massless,XM_massless,total
59,"[-1, 1, -3]","[1, 0]",7,"[5, 5, 7, -8, -12, -12, 15]",1,"[{'D→nu_R+fi+fj': [-12, 5, 7]}]",1,0,0,0,"{2: [5, -12]}","[-12, 5]",0,0,0,3,10,10,10,64


In [79]:
fl=fl[fl.apply(lambda r:min([r['DD_massless'],r['DM_massless'],r['XM_massless'],r['XD_massless']])<=2,axis='columns' )
     ].sort_values(['n',massless[mdl],'sltn_len','total'])[
       ['n','l','k','gcd','solution','DD', 'DM', 'XD', 'XM', 'DD_massless',
       'DM_massless','XD_massless', 'XM_massless','total','sltn_len','nu_R']].reset_index(drop=True)
fl

,n,l,k,gcd,solution,DD,DM,XD,XM,DD_massless,DM_massless,XD_massless,XM_massless,total,sltn_len,nu_R
0,6,"[2, -2]","[-1, 0]",4,"[1, -2, -3, 5, 5, -6]",[5],0,0,0,0,10,10,10,22,-1,"[{'D→nu_R+fi+fj': [-6, 1, 5]}, {'D→nu_R+fi+fj': [-3, -2, 5]}]"
1,7,"[-1, 0]","[-3, -4, -3]",6,"[1, 2, 2, -3, -3, -3, 4]",[-3],0,0,0,2,10,10,10,18,-2,"[{'D→nu_R+fi+fj': [-3, 1, 2]}]"
2,7,"[-4, -6]","[1, 0, -1]",3,"[1, 1, -3, -4, 6, 6, -7]","[1, 6]",[6],0,0,2,2,10,10,28,-2,"[{'D→nu_R+fi+fj': [-7, 1, 6]}, {'D→nu_R+2fi': [6, -3]}]"
3,7,"[-1, 1]","[2, -7, -6]",3,"[2, 3, 3, -4, -5, -6, 7]",[3],0,0,0,1,10,10,10,30,-1,"[{'D→nu_R+fi+fj': [-5, 2, 3]}]"
4,7,"[2, 0]","[-1, -4, -3]",3,"[1, 3, -4, 5, -6, -6, 7]",[-6],[-6],0,0,2,2,10,10,32,-1,"[{'D→nu_R+2fi': [-6, 3]}, {'D→nu_R+fi+fj': [-6, 1, 5]}]"
5,7,"[4, -5]","[3, 4, -4]",1,"[1, 1, -4, -4, 7, 8, -9]",[1],0,0,0,2,10,10,10,34,-1,"[{'D→nu_R+fi+fj': [-9, 1, 8]}]"
6,8,"[1, -6, -9]","[1, 0, -2]",2,"[1, 1, 2, 3, -4, -4, -5, 6]","[-4, 1]",[-4],0,0,2,3,10,10,26,-2,"[{'D→nu_R+fi+fj': [-4, 1, 3]}, {'D→nu_R+2fi': [-4, 2]}]"
7,8,"[1, 5, 1]","[1, 0, 1]",2,"[1, -2, -9, -9, 10, 11, 11, -13]","[-9, 11]",0,0,0,2,10,10,10,66,-2,"[{'D→nu_R+fi+fj': [-9, -2, 11]}]"
8,8,"[1, -2, 2]","[2, -1, 0]",1,"[1, 2, 2, 2, -3, -5, -6, 7]",[2],0,0,0,1,10,10,10,28,-1,"[{'D→nu_R+fi+fj': [-3, 1, 2]}]"
9,8,"[-1, -2, -1]","[-4, -2, -3]",2,"[1, 1, 2, -3, 4, -6, -7, 8]",[1],0,0,0,2,10,10,10,32,-1,"[{'D→nu_R+fi+fj': [-3, 1, 2]}]"


In [80]:
np.sort( fl['solution'].apply(lambda l: np.abs(l).max()).unique() )#.max()

array([ 4,  6,  7,  8,  9, 10, 11, 12, 13, 15, 17, 18, 20, 21, 22])

In [81]:
fl6=fl6[fl6.apply(lambda r:min([r['DD_massless'],r['DM_massless'],r['XM_massless'],r['XD_massless']])<=2,axis='columns' )
       ].sort_values(['n',massless[mdl],'sltn_len','total'])[
       ['n','l','k','gcd','solution','DD', 'DM', 'XD', 'XM', 'DD_massless',
       'DM_massless','XD_massless', 'XM_massless','total','sltn_len','nu_R']].reset_index(drop=True)

In [82]:
fl6

,n,l,k,gcd,solution,DD,DM,XD,XM,DD_massless,DM_massless,XD_massless,XM_massless,total,sltn_len,nu_R
0,6,"[-1, 1]","[4, -2]",3,"[1, 1, 1, -4, -4, 5]",0,[-4],0,0,10,1,10,10,16,-2,"[{'D→nu_R+4fi': [-4, 1]}]"
1,6,"[-4, 2]","[1, 6]",72,"[1, -4, -4, 9, 9, -11]",0,[-4],0,0,10,1,10,10,38,-1,"[{'D→nu_R+4fi': [-4, 1]}]"
2,6,"[2, 1]","[2, 1]",1,"[1, -4, -8, 14, 14, -17]",[14],0,0,0,2,10,10,10,58,-1,"[{'D→nu_R+2fi+2fj': [-8, 1, 14]}]"
3,7,"[4, -5]","[3, 4, -4]",1,"[1, 1, -4, -4, 7, 8, -9]",0,[-4],0,0,10,1,10,10,34,-2,"[{'D→nu_R+4fi': [-4, 1]}]"
4,7,"[3, 1]","[2, 3, -1]",1,"[3, 3, 3, -5, -5, -7, 8]",0,0,[-5],0,10,10,0,10,34,-1,"[{'X→nu_R+2fi+2fj': [-7, -5, 8]}]"
5,7,"[-3, 0]","[-1, 1, 2]",2,"[4, 4, 5, -7, -8, -9, 11]",[4],0,0,0,1,10,10,10,48,-1,"[{'D→nu_R+2fi+2fj': [-7, 4, 5]}]"
6,7,"[14, 12]","[-4, -1, 10]",176,"[7, 10, -18, 25, -28, -28, 32]",[-28],[-28],0,0,2,2,10,10,148,-1,"[{'D→nu_R+4fi': [-28, 7]}, {'D→nu_R+2fi+2fj': [-28, -18, 32]}]"
7,8,"[1, 2, -1]","[3, -1, 1]",2,"[1, 3, 3, 3, -5, -7, -7, 9]",0,0,0,[-7],10,10,10,2,38,-1,"[{'X→nu_R+4fi': [-7, 1]}]"
8,8,"[-1, 3, 2]","[7, 10, 13]",18,"[1, -7, -7, 17, 17, 19, -20, -20]",[-20],0,0,0,2,10,10,10,108,-3,"[{'D→nu_R+2fi+2fj': [-20, -7, 17]}]"
9,8,"[1, -6, -9]","[1, 0, -2]",2,"[1, 1, 2, 3, -4, -4, -5, 6]",0,[-4],0,0,10,2,10,10,26,-2,"[{'D→nu_R+4fi': [-4, 1]}]"


In [83]:
n=9
x=1
fl6[(fl6['n']==n) & (fl6['XD_massless']==x)].sort_values(['DD_massless','DM_massless','XM_massless'],ascending=False)

,n,l,k,gcd,solution,DD,DM,XD,XM,DD_massless,DM_massless,XD_massless,XM_massless,total,sltn_len,nu_R
37,9,"[4, 9, 3]","[2, 1, 3, 4]",27,"[4, 4, 4, 5, -6, -6, -6, -10, 11]",0,0,"[4, -6]",0,10,10,1,10,56,-1,"[{'X→nu_R+2fi+2fj': [-10, -6, 11]}, {'X→nu_R+2fi+2fj': [-10, 4, 11]}]"
43,9,"[18, 13, 6]","[-10, 6, 1, -4]",608,"[2, -7, -8, -8, -8, 18, 18, 18, -25]",0,[-8],"[-8, 18]",0,10,3,1,10,112,-1,"[{'D→nu_R+4fi': [-8, 2]}, {'X→nu_R+2fi+2fj': [-7, 2, 18]}, {'X→nu_R+2fi+2fj': [-8, -7, 2]}]"


In [84]:
np.sort( fl6['solution'].apply(lambda l: np.abs(l).max()).unique() ).max()

32

In [85]:
import re
def add_boldsymbol(ss):
    if str(ss).find(r'\boldsymbol')==-1:
        return re.sub('(\-*[0-9]+)',r'\\boldsymbol{\1}',str(ss))
    else:
        return ss

In [86]:
#kk=fl.copy()
kk=fl6.copy()
for i in kk.index:
#if True:
#    i=6
    for s in ['DD','DM','XD','XM']:
        if kk.loc[i,'{}_massless'.format(s)]==0:
            kk.loc[i,'n']=add_boldsymbol(kk.loc[i,'n'])
            kk.loc[i,s]  =add_boldsymbol(kk.loc[i,s])    
#kk=fl[['n','l','k','solution','n','DD','DM','XD','XM']]
kkk=kk[['n','l','k','solution','gcd','DD','DM','XD','XM']]
def f(x):
    return  r'{}'.format(str(x).replace('[','(').replace(']',')'))

kkk.to_latex('solutions.tex',index=False,formatters=dict( [(k,f) for k in kk.columns ]) ,escape=False  )

In [87]:
kk

,n,l,k,gcd,solution,DD,DM,XD,XM,DD_massless,DM_massless,XD_massless,XM_massless,total,sltn_len,nu_R
0,6,"[-1, 1]","[4, -2]",3,"[1, 1, 1, -4, -4, 5]",0,[-4],0,0,10,1,10,10,16,-2,"[{'D→nu_R+4fi': [-4, 1]}]"
1,6,"[-4, 2]","[1, 6]",72,"[1, -4, -4, 9, 9, -11]",0,[-4],0,0,10,1,10,10,38,-1,"[{'D→nu_R+4fi': [-4, 1]}]"
2,6,"[2, 1]","[2, 1]",1,"[1, -4, -8, 14, 14, -17]",[14],0,0,0,2,10,10,10,58,-1,"[{'D→nu_R+2fi+2fj': [-8, 1, 14]}]"
3,7,"[4, -5]","[3, 4, -4]",1,"[1, 1, -4, -4, 7, 8, -9]",0,[-4],0,0,10,1,10,10,34,-2,"[{'D→nu_R+4fi': [-4, 1]}]"
4,\boldsymbol{7},"[3, 1]","[2, 3, -1]",1,"[3, 3, 3, -5, -5, -7, 8]",0,0,[\boldsymbol{-5}],0,10,10,0,10,34,-1,"[{'X→nu_R+2fi+2fj': [-7, -5, 8]}]"
5,7,"[-3, 0]","[-1, 1, 2]",2,"[4, 4, 5, -7, -8, -9, 11]",[4],0,0,0,1,10,10,10,48,-1,"[{'D→nu_R+2fi+2fj': [-7, 4, 5]}]"
6,7,"[14, 12]","[-4, -1, 10]",176,"[7, 10, -18, 25, -28, -28, 32]",[-28],[-28],0,0,2,2,10,10,148,-1,"[{'D→nu_R+4fi': [-28, 7]}, {'D→nu_R+2fi+2fj': [-28, -18, 32]}]"
7,8,"[1, 2, -1]","[3, -1, 1]",2,"[1, 3, 3, 3, -5, -7, -7, 9]",0,0,0,[-7],10,10,10,2,38,-1,"[{'X→nu_R+4fi': [-7, 1]}]"
8,8,"[-1, 3, 2]","[7, 10, 13]",18,"[1, -7, -7, 17, 17, 19, -20, -20]",[-20],0,0,0,2,10,10,10,108,-3,"[{'D→nu_R+2fi+2fj': [-20, -7, 17]}]"
9,8,"[1, -6, -9]","[1, 0, -2]",2,"[1, 1, 2, 3, -4, -4, -5, 6]",0,[-4],0,0,10,2,10,10,26,-2,"[{'D→nu_R+4fi': [-4, 1]}]"


In [88]:
n=r'\boldsymbol{9}'
x=0
kk[(kk['n']==n) & (kk['DD_massless']==x)]

,n,l,k,gcd,solution,DD,DM,XD,XM,DD_massless,DM_massless,XD_massless,XM_massless,total,sltn_len,nu_R


## Manual analysis

In [89]:
fl['Q']=1
i=1
fl[fl['DD_massless']==i].sort_values(['n','DM_massless','XD_massless','XM_massless'])

,n,l,k,gcd,solution,DD,DM,XD,XM,DD_massless,DM_massless,XD_massless,XM_massless,total,sltn_len,nu_R,Q
3,7,"[-1, 1]","[2, -7, -6]",3,"[2, 3, 3, -4, -5, -6, 7]",[3],0,0,0,1,10,10,10,30,-1,"[{'D→nu_R+fi+fj': [-5, 2, 3]}]",1
13,8,"[4, 7, -1]","[12, -4, -6]",352,"[3, -4, -5, 8, 8, -11, -12, 13]",[8],[8],0,0,1,1,10,10,64,-1,"[{'D→nu_R+2fi': [8, -4]}, {'D→nu_R+fi+fj': [-11, 3, 8]}]",1
8,8,"[1, -2, 2]","[2, -1, 0]",1,"[1, 2, 2, 2, -3, -5, -6, 7]",[2],0,0,0,1,10,10,10,28,-1,"[{'D→nu_R+fi+fj': [-3, 1, 2]}]",1
29,9,"[-9, -4, 5]","[-1, -3, 3, 2]",36,"[1, -2, -2, -2, 5, -7, 8, 9, -10]",[-2],[-2],0,0,1,1,10,10,46,-1,"[{'D→nu_R+2fi': [-2, 1]}, {'D→nu_R+fi+fj': [-7, -2, 9]}]",1
25,9,"[-2, 3, 0]","[4, 8, 6, 3]",4,"[1, 1, 2, 2, 3, -5, -6, -6, 8]",[2],[-6],0,0,1,4,10,10,34,-1,"[{'D→nu_R+fi+fj': [-5, 2, 3]}, {'D→nu_R+2fi': [-6, 3]}]",1
33,9,"[9, 12, 7]","[3, 6, 1, -8]",349,"[1, 2, -6, -6, -6, 8, 9, 9, -11]",[9],0,[9],0,1,10,0,10,58,-1,"[{'D→nu_R+fi+fj': [-11, 2, 9]}, {'X→nu_R+fi+fj': [-11, 8, 9]}]",1
24,9,"[10, 16, 5]","[-5, 0, -10, -7]",605,"[2, -5, -5, -7, -10, 15, 15, 16, -21]","[-5, 15]",0,0,0,1,10,10,10,96,-2,"[{'D→nu_R+fi+fj': [-10, -5, 15]}]",1
30,9,"[-2, -3, -2]","[-1, 0, 1, -1]",1,"[2, 2, -3, 5, -6, 7, -8, -9, 10]",[2],0,0,0,1,10,10,10,52,-1,"[{'D→nu_R+fi+fj': [-9, 2, 7]}]",1


In [90]:
fl[fl['DM_massless']==i].sort_values(['n','DD_massless','XD_massless','XM_massless'])

,n,l,k,gcd,solution,DD,DM,XD,XM,DD_massless,DM_massless,XD_massless,XM_massless,total,sltn_len,nu_R,Q
13,8,"[4, 7, -1]","[12, -4, -6]",352,"[3, -4, -5, 8, 8, -11, -12, 13]",[8],[8],0,0,1,1,10,10,64,-1,"[{'D→nu_R+2fi': [8, -4]}, {'D→nu_R+fi+fj': [-11, 3, 8]}]",1
29,9,"[-9, -4, 5]","[-1, -3, 3, 2]",36,"[1, -2, -2, -2, 5, -7, 8, 9, -10]",[-2],[-2],0,0,1,1,10,10,46,-1,"[{'D→nu_R+2fi': [-2, 1]}, {'D→nu_R+fi+fj': [-7, -2, 9]}]",1
35,9,"[-2, -4, 7]","[2, -4, -2, 8]",20,"[1, -2, -2, -3, -3, -3, 14, 20, -22]",0,[-2],0,0,10,1,10,10,70,-1,"[{'D→nu_R+2fi': [-2, 1]}]",1


In [91]:
fl.loc[14,'Q']=0
fl.loc[10,'Q']=0
fl[fl['XD_massless']==i].sort_values(['n','DD_massless','DM_massless','XM_massless'])

,n,l,k,gcd,solution,DD,DM,XD,XM,DD_massless,DM_massless,XD_massless,XM_massless,total,sltn_len,nu_R,Q
12,8,"[5, 0, -5]","[10, -5, 5]",625,"[2, -5, -5, -5, 7, 8, 8, -10]",[8],0,[8],0,2,10,1,10,50,-1,"[{'D→nu_R+fi+fj': [-10, 2, 8]}, {'X→nu_R+fi+fj': [-10, 7, 8]}]",1
14,8,"[10, 4, -6]","[10, -8, 4]",2112,"[2, -5, -7, -7, 10, 10, 10, -13]",0,[10],[-7],0,10,3,1,10,64,-1,"[{'D→nu_R+2fi': [10, -5]}, {'X→nu_R+fi+fj': [-7, -5, 2]}]",0
10,8,"[1, 2, -1]","[3, -1, 1]",2,"[1, 3, 3, 3, -5, -7, -7, 9]",0,0,[-7],0,10,10,1,10,38,-1,"[{'X→nu_R+fi+fj': [-7, -5, 9]}]",0
34,9,"[4, -10, 9]","[6, -6, -4, 12]",732,"[4, 4, 6, 6, -7, -7, -7, -12, 13]",0,0,[6],0,10,10,1,10,66,-1,"[{'X→nu_R+fi+fj': [-12, 6, 13]}]",1


In [92]:
fl[fl['XM_massless']==i].sort_values(['n','DD_massless','DM_massless','XD_massless'])

,n,l,k,gcd,solution,DD,DM,XD,XM,DD_massless,DM_massless,XD_massless,XM_massless,total,sltn_len,nu_R,Q


In [93]:
i=2
fl.loc[7,'Q']=0
fl[fl['DD_massless']==i].sort_values(['n','DM_massless','XD_massless','XM_massless'])

,n,l,k,gcd,solution,DD,DM,XD,XM,DD_massless,DM_massless,XD_massless,XM_massless,total,sltn_len,nu_R,Q
2,7,"[-4, -6]","[1, 0, -1]",3,"[1, 1, -3, -4, 6, 6, -7]","[1, 6]",[6],0,0,2,2,10,10,28,-2,"[{'D→nu_R+fi+fj': [-7, 1, 6]}, {'D→nu_R+2fi': [6, -3]}]",1
4,7,"[2, 0]","[-1, -4, -3]",3,"[1, 3, -4, 5, -6, -6, 7]",[-6],[-6],0,0,2,2,10,10,32,-1,"[{'D→nu_R+2fi': [-6, 3]}, {'D→nu_R+fi+fj': [-6, 1, 5]}]",1
1,7,"[-1, 0]","[-3, -4, -3]",6,"[1, 2, 2, -3, -3, -3, 4]",[-3],0,0,0,2,10,10,10,18,-2,"[{'D→nu_R+fi+fj': [-3, 1, 2]}]",1
5,7,"[4, -5]","[3, 4, -4]",1,"[1, 1, -4, -4, 7, 8, -9]",[1],0,0,0,2,10,10,10,34,-1,"[{'D→nu_R+fi+fj': [-9, 1, 8]}]",1
11,8,"[1, -2, 1]","[3, -1, 0]",1,"[1, 2, 3, 5, -6, -6, -9, 10]",[-6],[-6],0,0,2,2,10,10,42,-1,"[{'D→nu_R+2fi': [-6, 3]}, {'D→nu_R+fi+fj': [-6, 1, 5]}]",1
6,8,"[1, -6, -9]","[1, 0, -2]",2,"[1, 1, 2, 3, -4, -4, -5, 6]","[-4, 1]",[-4],0,0,2,3,10,10,26,-2,"[{'D→nu_R+fi+fj': [-4, 1, 3]}, {'D→nu_R+2fi': [-4, 2]}]",1
12,8,"[5, 0, -5]","[10, -5, 5]",625,"[2, -5, -5, -5, 7, 8, 8, -10]",[8],0,[8],0,2,10,1,10,50,-1,"[{'D→nu_R+fi+fj': [-10, 2, 8]}, {'X→nu_R+fi+fj': [-10, 7, 8]}]",1
7,8,"[1, 5, 1]","[1, 0, 1]",2,"[1, -2, -9, -9, 10, 11, 11, -13]","[-9, 11]",0,0,0,2,10,10,10,66,-2,"[{'D→nu_R+fi+fj': [-9, -2, 11]}]",0
9,8,"[-1, -2, -1]","[-4, -2, -3]",2,"[1, 1, 2, -3, 4, -6, -7, 8]",[1],0,0,0,2,10,10,10,32,-1,"[{'D→nu_R+fi+fj': [-3, 1, 2]}]",1
20,9,"[10, 6, -8]","[-2, -5, 8, -7]",232,"[1, 1, -5, -7, 12, 14, 14, -15, -15]","[-15, 1, 14]",[14],0,0,2,2,10,10,84,-3,"[{'D→nu_R+fi+fj': [-15, 1, 14]}, {'D→nu_R+2fi': [14, -7]}]",1


In [94]:
fl.loc[15,'Q']=0
fl.loc[39,'Q']=0
fl[fl['DM_massless']==i].sort_values(['n','DD_massless','XD_massless','XM_massless'])

,n,l,k,gcd,solution,DD,DM,XD,XM,DD_massless,DM_massless,XD_massless,XM_massless,total,sltn_len,nu_R,Q
2,7.0,"[-4, -6]","[1, 0, -1]",3.0,"[1, 1, -3, -4, 6, 6, -7]","[1, 6]",[6],0,0,2.0,2.0,10.0,10.0,28.0,-2.0,"[{'D→nu_R+fi+fj': [-7, 1, 6]}, {'D→nu_R+2fi': [6, -3]}]",1.0
4,7.0,"[2, 0]","[-1, -4, -3]",3.0,"[1, 3, -4, 5, -6, -6, 7]",[-6],[-6],0,0,2.0,2.0,10.0,10.0,32.0,-1.0,"[{'D→nu_R+2fi': [-6, 3]}, {'D→nu_R+fi+fj': [-6, 1, 5]}]",1.0
11,8.0,"[1, -2, 1]","[3, -1, 0]",1.0,"[1, 2, 3, 5, -6, -6, -9, 10]",[-6],[-6],0,0,2.0,2.0,10.0,10.0,42.0,-1.0,"[{'D→nu_R+2fi': [-6, 3]}, {'D→nu_R+fi+fj': [-6, 1, 5]}]",1.0
15,8.0,"[4, 2, 13]","[-2, 8, 12]",1264.0,"[4, -8, -8, 11, -12, 14, 21, -22]",0,[-8],0,0,10.0,2.0,10.0,10.0,100.0,-1.0,"[{'D→nu_R+2fi': [-8, 4]}]",0.0
20,9.0,"[10, 6, -8]","[-2, -5, 8, -7]",232.0,"[1, 1, -5, -7, 12, 14, 14, -15, -15]","[-15, 1, 14]",[14],0,0,2.0,2.0,10.0,10.0,84.0,-3.0,"[{'D→nu_R+fi+fj': [-15, 1, 14]}, {'D→nu_R+2fi': [14, -7]}]",1.0
23,9.0,"[3, 0, 2]","[-2, 13, -5, -4]",86.0,"[1, -2, -2, -4, 7, -9, 11, 11, -13]","[-2, 11]",[-2],0,0,2.0,2.0,10.0,10.0,60.0,-2.0,"[{'D→nu_R+2fi': [-2, 1]}, {'D→nu_R+fi+fj': [-9, -2, 11]}]",1.0
32,9.0,"[-2, 2, 4]","[-1, -8, -6, -7]",1.0,"[1, -2, 3, 5, -6, 7, -10, -10, 12]",[-10],[-10],0,0,2.0,2.0,10.0,10.0,56.0,-1.0,"[{'D→nu_R+2fi': [-10, 5]}, {'D→nu_R+fi+fj': [-10, -2, 12]}, {'D→nu_R+fi+fj': [-10, 3, 7]}]",1.0
37,9.0,"[18, 12, 3]","[-6, 15, 3, -3]",2268.0,"[1, -2, -2, 5, -7, -7, 14, 18, -20]",0,[-2],0,0,10.0,2.0,10.0,10.0,76.0,-1.0,"[{'D→nu_R+2fi': [-2, 1]}]",1.0


In [96]:
fl.loc[38,'Q']=0
fl.loc[31,'Q']=0
fl[fl['XD_massless']==i].sort_values(['n','DD_massless','DM_massless','XM_massless'])

,n,l,k,gcd,solution,DD,DM,XD,XM,DD_massless,DM_massless,XD_massless,XM_massless,total,sltn_len,nu_R,Q
38,9.0,"[-5, -1, -2]","[-1, -6, 0, -6]",4.0,"[1, -2, -2, -2, 5, -16, 17, 17, -18]",[17],[-2],[17],0,3.0,4.0,2.0,10.0,80.0,-1.0,"[{'D→nu_R+fi+fj': [-18, 1, 17]}, {'D→nu_R+2fi': [-2, 1]}, {'X→nu_R+fi+fj': [-16, 1, 17]}]",0.0
28,9.0,"[7, 1, 6]","[-2, 8, 2, 4]",296.0,"[2, -3, 4, 4, 4, -6, -7, -7, 9]",[4],0,[-7],0,3.0,10.0,2.0,10.0,46.0,-1.0,"[{'D→nu_R+fi+fj': [-6, 2, 4]}, {'X→nu_R+fi+fj': [-7, -6, 9]}]",1.0
21,9.0,"[3, -1, 0]","[1, -3, -4, -5]",4.0,"[2, 2, 2, -3, -3, 4, -5, -5, 6]",0,0,"[-5, -3]",0,10.0,10.0,2.0,10.0,32.0,-2.0,"[{'X→nu_R+fi+fj': [-5, -3, 6]}]",1.0
31,9.0,"[12, 21, 13]","[9, 12, 8, 1]",688.0,"[2, -3, -3, -3, -6, 7, 7, 11, -12]",0,0,[7],0,10.0,10.0,2.0,10.0,54.0,-1.0,"[{'X→nu_R+fi+fj': [-6, 2, 7]}]",0.0


In [97]:
fl.loc[17,'Q']=0
fl[fl['XM_massless']==i].sort_values(['n','DD_massless','DM_massless','XD_massless'])

,n,l,k,gcd,solution,DD,DM,XD,XM,DD_massless,DM_massless,XD_massless,XM_massless,total,sltn_len,nu_R,Q
16,9.0,"[-9, 5, 1]","[-1, 2, 0, -2]",2.0,"[1, 1, -4, -5, 9, 9, 9, -10, -10]",[9],0,0,[1],0.0,10.0,10.0,2.0,58.0,-3.0,"[{'D→nu_R+fi+fj': [-10, 1, 9]}, {'D→nu_R+fi+fj': [-5, -4, 9]}, {'X→nu_R+2fi': [1, -5]}]",1.0
17,9.0,"[8, 7, 0]","[-1, 13, 15, 13]",620.0,"[1, 13, 13, -15, -15, -15, -16, 17, 17]",0,0,0,[13],10.0,10.0,10.0,2.0,122.0,-1.0,"[{'X→nu_R+2fi': [13, 1]}]",0.0


In [98]:
fl.value_counts('Q')

Q  
1.0    31
0.0    10
dtype: int64

D-6

In [99]:
fl6['Q']=1
i=1
fl6[fl6['DD_massless']==i].sort_values(['n','DM_massless','XD_massless','XM_massless'])

,n,l,k,gcd,solution,DD,DM,XD,XM,DD_massless,DM_massless,XD_massless,XM_massless,total,sltn_len,nu_R,Q
5,7,"[-3, 0]","[-1, 1, 2]",2,"[4, 4, 5, -7, -8, -9, 11]",[4],0,0,0,1,10,10,10,48,-1,"[{'D→nu_R+2fi+2fj': [-7, 4, 5]}]",1
17,8,"[3, 5, 2]","[-2, -5, -4]",2,"[2, -3, 7, -8, -8, 11, 14, -15]",[-8],[-8],0,0,1,1,10,10,68,-1,"[{'D→nu_R+4fi': [-8, 2]}, {'D→nu_R+2fi+2fj': [-8, -3, 7]}]",1
14,8,"[1, 0, -3]","[1, 2, -2]",2,"[1, 4, 5, 5, -8, -10, -10, 13]",[-10],0,0,0,1,10,10,10,56,-1,"[{'D→nu_R+2fi+2fj': [-10, -8, 13]}, {'D→nu_R+2fi+2fj': [-10, 1, 4]}]",1
42,9,"[-10, 5, 10]","[-5, 3, 2, -6]",300,"[3, 3, -8, -12, -12, 13, 14, 18, -19]",[-12],[-12],0,0,1,1,10,10,102,-1,"[{'D→nu_R+4fi': [-12, 3]}, {'D→nu_R+2fi+2fj': [-12, -8, 14]}]",1
44,9,"[-6, 9, -3]","[12, -6, 12, -3]",2268,"[1, -3, -4, -4, 10, -21, 23, 24, -26]",[-4],[-4],0,0,1,1,10,10,116,-1,"[{'D→nu_R+4fi': [-4, 1]}, {'D→nu_R+2fi+2fj': [-21, -4, 23]}]",1
23,9,"[-2, -1, 0]","[1, 4, -4, -5]",2,"[1, -5, 8, 8, 8, -11, -12, -12, 15]",[8],0,0,[-12],1,10,10,3,80,-1,"[{'D→nu_R+2fi+2fj': [-5, 1, 8]}, {'X→nu_R+4fi': [-12, 1]}]",1
25,9,"[18, 9, 10]","[1, -8, 14, 9]",1790,"[4, 8, 8, -9, -13, -17, -17, 18, 18]",[18],0,0,0,1,10,10,10,112,-3,"[{'D→nu_R+2fi+2fj': [-17, 8, 18]}, {'D→nu_R+2fi+2fj': [-13, 4, 18]}]",1
26,9,"[3, -1, 0]","[1, -3, -4, -5]",4,"[2, 2, 2, -3, -3, 4, -5, -5, 6]",[2],0,0,0,1,10,10,10,32,-2,"[{'D→nu_R+2fi+2fj': [-5, 2, 4]}]",1
36,9,"[-2, -1, 0]","[1, 3, 1, -4]",1,"[1, -3, 4, 4, -6, 8, -9, -10, 11]",[4],0,0,0,1,10,10,10,56,-1,"[{'D→nu_R+2fi+2fj': [-10, 4, 8]}, {'D→nu_R+2fi+2fj': [-3, 1, 4]}]",1


In [100]:
fl6[fl6['DM_massless']==i].sort_values(['n','DD_massless','XD_massless','XM_massless'])

,n,l,k,gcd,solution,DD,DM,XD,XM,DD_massless,DM_massless,XD_massless,XM_massless,total,sltn_len,nu_R,Q
0,6,"[-1, 1]","[4, -2]",3,"[1, 1, 1, -4, -4, 5]",0,[-4],0,0,10,1,10,10,16,-2,"[{'D→nu_R+4fi': [-4, 1]}]",1
1,6,"[-4, 2]","[1, 6]",72,"[1, -4, -4, 9, 9, -11]",0,[-4],0,0,10,1,10,10,38,-1,"[{'D→nu_R+4fi': [-4, 1]}]",1
3,7,"[4, -5]","[3, 4, -4]",1,"[1, 1, -4, -4, 7, 8, -9]",0,[-4],0,0,10,1,10,10,34,-2,"[{'D→nu_R+4fi': [-4, 1]}]",1
17,8,"[3, 5, 2]","[-2, -5, -4]",2,"[2, -3, 7, -8, -8, 11, 14, -15]",[-8],[-8],0,0,1,1,10,10,68,-1,"[{'D→nu_R+4fi': [-8, 2]}, {'D→nu_R+2fi+2fj': [-8, -3, 7]}]",1
11,8,"[4, 0, 16]","[-1, 15, 4]",1792,"[1, 1, -4, -4, -4, 12, 15, -17]",0,[-4],0,0,10,1,10,10,58,-2,"[{'D→nu_R+4fi': [-4, 1]}]",1
42,9,"[-10, 5, 10]","[-5, 3, 2, -6]",300,"[3, 3, -8, -12, -12, 13, 14, 18, -19]",[-12],[-12],0,0,1,1,10,10,102,-1,"[{'D→nu_R+4fi': [-12, 3]}, {'D→nu_R+2fi+2fj': [-12, -8, 14]}]",1
44,9,"[-6, 9, -3]","[12, -6, 12, -3]",2268,"[1, -3, -4, -4, 10, -21, 23, 24, -26]",[-4],[-4],0,0,1,1,10,10,116,-1,"[{'D→nu_R+4fi': [-4, 1]}, {'D→nu_R+2fi+2fj': [-21, -4, 23]}]",1
41,9,"[-4, -3, 3]","[-8, -5, -9, -4]",183,"[1, -4, -4, -4, 12, -14, 15, 18, -20]",0,[-4],0,0,10,1,10,10,92,-1,"[{'D→nu_R+4fi': [-4, 1]}]",1


In [101]:
#fl.loc[14,'Q']=0
#fl.loc[10,'Q']=0
fl6[fl6['XD_massless']==i].sort_values(['n','DD_massless','DM_massless','XM_massless'])

,n,l,k,gcd,solution,DD,DM,XD,XM,DD_massless,DM_massless,XD_massless,XM_massless,total,sltn_len,nu_R,Q
19,8,"[17, 0, 17]","[3, 7, 3]",4624,"[3, 3, 3, -7, 17, -23, -23, 27]",0,0,[-23],0,10,10,1,10,106,-1,"[{'X→nu_R+2fi+2fj': [-23, -7, 17]}]",1
43,9,"[18, 13, 6]","[-10, 6, 1, -4]",608,"[2, -7, -8, -8, -8, 18, 18, 18, -25]",0,[-8],"[-8, 18]",0,10,3,1,10,112,-1,"[{'D→nu_R+4fi': [-8, 2]}, {'X→nu_R+2fi+2fj': [-7, 2, 18]}, {'X→nu_R+2fi+2fj': [-8, -7, 2]}]",1
37,9,"[4, 9, 3]","[2, 1, 3, 4]",27,"[4, 4, 4, 5, -6, -6, -6, -10, 11]",0,0,"[4, -6]",0,10,10,1,10,56,-1,"[{'X→nu_R+2fi+2fj': [-10, -6, 11]}, {'X→nu_R+2fi+2fj': [-10, 4, 11]}]",1


In [102]:
fl6[fl6['XM_massless']==i].sort_values(['n','DD_massless','DM_massless','XD_massless'])

,n,l,k,gcd,solution,DD,DM,XD,XM,DD_massless,DM_massless,XD_massless,XM_massless,total,sltn_len,nu_R,Q


In [103]:
i=2
fl.loc[7,'Q']=0
fl[fl['DD_massless']==i].sort_values(['n','DM_massless','XD_massless','XM_massless'])

,n,l,k,gcd,solution,DD,DM,XD,XM,DD_massless,DM_massless,XD_massless,XM_massless,total,sltn_len,nu_R,Q
2,7.0,"[-4, -6]","[1, 0, -1]",3.0,"[1, 1, -3, -4, 6, 6, -7]","[1, 6]",[6],0,0,2.0,2.0,10.0,10.0,28.0,-2.0,"[{'D→nu_R+fi+fj': [-7, 1, 6]}, {'D→nu_R+2fi': [6, -3]}]",1.0
4,7.0,"[2, 0]","[-1, -4, -3]",3.0,"[1, 3, -4, 5, -6, -6, 7]",[-6],[-6],0,0,2.0,2.0,10.0,10.0,32.0,-1.0,"[{'D→nu_R+2fi': [-6, 3]}, {'D→nu_R+fi+fj': [-6, 1, 5]}]",1.0
1,7.0,"[-1, 0]","[-3, -4, -3]",6.0,"[1, 2, 2, -3, -3, -3, 4]",[-3],0,0,0,2.0,10.0,10.0,10.0,18.0,-2.0,"[{'D→nu_R+fi+fj': [-3, 1, 2]}]",1.0
5,7.0,"[4, -5]","[3, 4, -4]",1.0,"[1, 1, -4, -4, 7, 8, -9]",[1],0,0,0,2.0,10.0,10.0,10.0,34.0,-1.0,"[{'D→nu_R+fi+fj': [-9, 1, 8]}]",1.0
11,8.0,"[1, -2, 1]","[3, -1, 0]",1.0,"[1, 2, 3, 5, -6, -6, -9, 10]",[-6],[-6],0,0,2.0,2.0,10.0,10.0,42.0,-1.0,"[{'D→nu_R+2fi': [-6, 3]}, {'D→nu_R+fi+fj': [-6, 1, 5]}]",1.0
6,8.0,"[1, -6, -9]","[1, 0, -2]",2.0,"[1, 1, 2, 3, -4, -4, -5, 6]","[-4, 1]",[-4],0,0,2.0,3.0,10.0,10.0,26.0,-2.0,"[{'D→nu_R+fi+fj': [-4, 1, 3]}, {'D→nu_R+2fi': [-4, 2]}]",1.0
12,8.0,"[5, 0, -5]","[10, -5, 5]",625.0,"[2, -5, -5, -5, 7, 8, 8, -10]",[8],0,[8],0,2.0,10.0,1.0,10.0,50.0,-1.0,"[{'D→nu_R+fi+fj': [-10, 2, 8]}, {'X→nu_R+fi+fj': [-10, 7, 8]}]",1.0
7,8.0,"[1, 5, 1]","[1, 0, 1]",2.0,"[1, -2, -9, -9, 10, 11, 11, -13]","[-9, 11]",0,0,0,2.0,10.0,10.0,10.0,66.0,-2.0,"[{'D→nu_R+fi+fj': [-9, -2, 11]}]",0.0
9,8.0,"[-1, -2, -1]","[-4, -2, -3]",2.0,"[1, 1, 2, -3, 4, -6, -7, 8]",[1],0,0,0,2.0,10.0,10.0,10.0,32.0,-1.0,"[{'D→nu_R+fi+fj': [-3, 1, 2]}]",1.0
20,9.0,"[10, 6, -8]","[-2, -5, 8, -7]",232.0,"[1, 1, -5, -7, 12, 14, 14, -15, -15]","[-15, 1, 14]",[14],0,0,2.0,2.0,10.0,10.0,84.0,-3.0,"[{'D→nu_R+fi+fj': [-15, 1, 14]}, {'D→nu_R+2fi': [14, -7]}]",1.0


In [104]:
#fl6.loc[15,'Q']=0
#fl6.loc[39,'Q']=0
fl6[fl6['DM_massless']==i].sort_values(['n','DD_massless','XD_massless','XM_massless'])

,n,l,k,gcd,solution,DD,DM,XD,XM,DD_massless,DM_massless,XD_massless,XM_massless,total,sltn_len,nu_R,Q
6,7,"[14, 12]","[-4, -1, 10]",176,"[7, 10, -18, 25, -28, -28, 32]",[-28],[-28],0,0,2,2,10,10,148,-1,"[{'D→nu_R+4fi': [-28, 7]}, {'D→nu_R+2fi+2fj': [-28, -18, 32]}]",1
15,8,"[-3, -1, 3]","[12, 14, 4]",1,"[1, -3, -3, 5, -11, 12, 12, -13]",[12],[12],0,0,2,2,10,10,60,-1,"[{'D→nu_R+4fi': [12, -3]}, {'D→nu_R+2fi+2fj': [-11, 5, 12]}]",1
20,8,"[-4, 17, 21]","[-6, 4, 8]",112,"[1, -5, -11, 15, -16, 20, 20, -24]",[20],[20],0,0,2,2,10,10,112,-1,"[{'D→nu_R+4fi': [20, -5]}, {'D→nu_R+2fi+2fj': [-11, 1, 20]}]",1
16,8,"[16, -1, -4]","[8, 14, -3]",1311,"[1, 2, 2, -8, -8, 12, 15, -16]",[2],[-8],0,0,3,2,10,10,64,-1,"[{'D→nu_R+2fi+2fj': [-16, 2, 15]}, {'D→nu_R+4fi': [-8, 2]}]",1
12,8,"[-8, -6, -9]","[2, 11, -8]",594,"[4, 4, 7, 14, -16, -16, -22, 25]",[4],[-16],0,0,4,2,10,10,108,-2,"[{'D→nu_R+2fi+2fj': [-16, 4, 14]}, {'D→nu_R+4fi': [-16, 4]}]",1
9,8,"[1, -6, -9]","[1, 0, -2]",2,"[1, 1, 2, 3, -4, -4, -5, 6]",0,[-4],0,0,10,2,10,10,26,-2,"[{'D→nu_R+4fi': [-4, 1]}]",1
18,8,"[4, 6, 12]","[4, 5, -6]",92,"[1, -2, -4, -4, -4, 15, 22, -24]",0,[-4],0,0,10,2,10,10,76,-1,"[{'D→nu_R+4fi': [-4, 1]}]",1
32,9,"[18, 15, 0]","[-2, 1, -1, -4]",3,"[5, 5, 5, -7, -8, 18, -20, -20, 22]",[-20],[-20],0,0,2,2,10,10,110,-2,"[{'D→nu_R+4fi': [-20, 5]}, {'D→nu_R+2fi+2fj': [-20, -8, 18]}]",1
38,9,"[3, 2, 3]","[-1, -3, -1, 1]",1,"[1, -2, -3, -6, -7, 8, 12, 12, -15]",[12],[12],0,0,2,2,10,10,66,-1,"[{'D→nu_R+4fi': [12, -3]}, {'D→nu_R+2fi+2fj': [-7, 1, 12]}]",1
40,9,"[-4, -8, 5]","[2, 4, -2, 9]",18,"[1, -2, -2, -4, -4, -4, 17, 27, -29]",0,[-4],0,0,10,2,10,10,90,-1,"[{'D→nu_R+4fi': [-4, 1]}]",1


In [105]:
#fl.loc[40,'Q']=0
#fl.loc[33,'Q']=0
fl6[fl6['XD_massless']==i].sort_values(['n','DD_massless','DM_massless','XM_massless'])

,n,l,k,gcd,solution,DD,DM,XD,XM,DD_massless,DM_massless,XD_massless,XM_massless,total,sltn_len,nu_R,Q
28,9,"[10, -5, -7]","[2, -8, -6, -2]",128,"[1, 1, 1, 4, -9, -10, -10, 11, 11]",[-10],0,"[-10, 11]",0,2,10,2,10,58,-2,"[{'D→nu_R+2fi+2fj': [-10, 1, 4]}, {'X→nu_R+2fi+2fj': [-10, 4, 11]}]",1
39,9,"[8, 16, -4]","[12, 14, 10, 6]",1280,"[3, 4, -10, -10, -10, 12, 12, 13, -14]",[12],0,[12],0,2,10,2,10,88,-1,"[{'D→nu_R+2fi+2fj': [-10, 4, 12]}, {'X→nu_R+2fi+2fj': [-14, 12, 13]}]",1
33,9,"[7, 0, 2]","[4, -4, 5, 2]",46,"[4, 7, -8, 9, -16, -16, -16, 18, 18]",[18],[-16],[18],0,3,4,2,10,112,-2,"[{'D→nu_R+2fi+2fj': [-16, 7, 18]}, {'D→nu_R+4fi': [-16, 4]}, {'X→nu_R+2fi+2fj': [-8, 7, 18]}]",1
29,9,"[-2, 1, -3]","[-1, 5, 2, 11]",16,"[3, 3, 3, -4, -4, 8, -11, -11, 13]",[-4],0,"[-11, -4]",0,3,10,2,10,60,-2,"[{'D→nu_R+2fi+2fj': [-11, -4, 13]}, {'X→nu_R+2fi+2fj': [-11, -4, 8]}]",1
30,9,"[16, -1, -6]","[1, 5, -5, -1]",32,"[3, -4, -4, -9, -13, 16, 16, 16, -21]",0,[16],[-4],0,10,4,2,10,102,-2,"[{'D→nu_R+4fi': [16, -4]}, {'X→nu_R+2fi+2fj': [-9, -4, 3]}]",1
31,9,"[7, 9, 11]","[8, -2, 1, 5]",67,"[5, 7, 7, -8, -15, -15, -15, 17, 17]",0,0,"[7, 17]",0,10,10,2,10,106,-2,"[{'X→nu_R+2fi+2fj': [-8, 7, 17]}]",1
35,9,"[-1, 2, 1]","[-1, 13, 0, 5]",47,"[2, -3, -3, -3, -5, 7, 7, 8, -10]",0,0,[7],0,10,10,2,10,48,-1,"[{'X→nu_R+2fi+2fj': [-10, 7, 8]}]",1


In [106]:
#fl.loc[17,'Q']=0
fl6[fl6['XM_massless']==i].sort_values(['n','DD_massless','DM_massless','XD_massless'])

,n,l,k,gcd,solution,DD,DM,XD,XM,DD_massless,DM_massless,XD_massless,XM_massless,total,sltn_len,nu_R,Q
7,8,"[1, 2, -1]","[3, -1, 1]",2,"[1, 3, 3, 3, -5, -7, -7, 9]",0,0,0,[-7],10,10,10,2,38,-1,"[{'X→nu_R+4fi': [-7, 1]}]",1
22,9,"[-4, -3, -2]","[-6, -7, 11, 5]",249,"[1, -4, 5, 5, -9, -9, -9, 10, 10]",0,0,0,[5],10,10,10,2,62,-1,"[{'X→nu_R+4fi': [5, 1]}]",1


In [107]:
fl.value_counts('Q')

Q  
1.0    31
0.0    10
dtype: int64